In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, MultiPolygon
import multiprocessing
import numpy as np

In [2]:
df_crashes = pd.read_csv(r'..\..\data\Motor_Vehicle_Collisions_-_Crashes.csv')
#df_precincts = pd.read_csv(r'..\..\data\nypp.csv')
df_precincts =  gpd.read_file('..\..\data\Police Precincts.geojson')

c:\users\alexf\anaconda3\envs\nyc_crash_env\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_crashes['crash_year'] = pd.DatetimeIndex(df_crashes['CRASH DATE']).year
df_crashes['crash_month'] = pd.DatetimeIndex(df_crashes['CRASH DATE']).month
df_crashes['crash_month_year'] = pd.to_datetime(df_crashes['CRASH DATE']).dt.to_period('M')
df_crashes['crash_day'] = pd.DatetimeIndex(df_crashes['CRASH DATE']).day
df_crashes['crash_hour'] = pd.DatetimeIndex(df_crashes['CRASH TIME']).hour

In [4]:
#too much effort to convert the missing latitude and longitude values from address
#grab the ones that are not null...
df_crashes_longitude_latitude_not_null = df_crashes.loc[
    (df_crashes['LONGITUDE'].notnull()) & (df_crashes['LATITUDE'].notnull()) 
    & (df_crashes['LONGITUDE'].notna()) & (df_crashes['LATITUDE'].notna())
]

In [5]:
df_crashes_longitude_latitude_not_null

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,crash_year,crash_month,crash_month_year,crash_day,crash_hour
0,06/09/2019,11:32,MANHATTAN,10012,40.725210,-73.995860,POINT (-73.99586 40.72521),EAST HOUSTON STREET,CROSBY STREET,NaN,...,Sedan,Taxi,NaN,NaN,NaN,2019,6,2019-06,9,11
1,06/28/2019,8:55,BROOKLYN,11235,40.586680,-73.945114,POINT (-73.945114 40.58668),VOORHIES AVENUE,EAST 24 STREET,NaN,...,Sedan,NaN,NaN,NaN,NaN,2019,6,2019-06,28,8
2,06/18/2019,15:00,BROOKLYN,11232,40.670570,-73.997630,POINT (-73.99763 40.67057),NaN,NaN,435 HAMILTON AVENUE,...,Sedan,Pick-up Truck,NaN,NaN,NaN,2019,6,2019-06,18,15
3,06/30/2019,22:30,NaN,NaN,40.617893,-74.164130,POINT (-74.16413 40.617893),GAULDY AVENUE,LONGDALE STREET,NaN,...,Sedan,NaN,NaN,NaN,NaN,2019,6,2019-06,30,22
4,06/10/2019,19:20,BROOKLYN,11208,40.681355,-73.880220,POINT (-73.88022 40.681355),FULTON STREET,HALE AVENUE,NaN,...,Sedan,Pick-up Truck,NaN,NaN,NaN,2019,6,2019-06,10,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1643286,09/18/2012,18:15,BROOKLYN,11234,40.626073,-73.923740,POINT (-73.9237396 40.6260734),AVENUE K,EAST 54 STREET,NaN,...,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN,2012,9,2012-09,18,18
1643287,09/22/2012,9:00,MANHATTAN,10012,40.720626,-73.994971,POINT (-73.9949711 40.7206259),ELIZABETH STREET,KENMARE STREET,NaN,...,PASSENGER VEHICLE,PASSENGER VEHICLE,NaN,NaN,NaN,2012,9,2012-09,22,9
1643288,09/20/2012,17:15,BROOKLYN,11207,40.684513,-73.909380,POINT (-73.9093801 40.6845131),BUSHWICK AVENUE,MOFFAT STREET,NaN,...,VAN,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,2012,9,2012-09,20,17
1643289,09/19/2012,12:36,MANHATTAN,10026,40.805032,-73.956820,POINT (-73.9568204 40.8050324),MANHATTAN AVENUE,WEST 116 STREET,NaN,...,TAXI,TAXI,NaN,NaN,NaN,2012,9,2012-09,19,12


In [6]:
df_crashes_longitude_latitude_not_null['LOCATION']

0              POINT (-73.99586 40.72521)
1             POINT (-73.945114 40.58668)
2              POINT (-73.99763 40.67057)
3             POINT (-74.16413 40.617893)
4             POINT (-73.88022 40.681355)
                        ...              
1643286    POINT (-73.9237396 40.6260734)
1643287    POINT (-73.9949711 40.7206259)
1643288    POINT (-73.9093801 40.6845131)
1643289    POINT (-73.9568204 40.8050324)
1643291    POINT (-73.9875391 40.7802028)
Name: LOCATION, Length: 1443867, dtype: object

In [7]:
df_crashes_longitude_latitude_not_null['LOCATION'][0]

'POINT (-73.99586 40.72521)'

In [8]:
df_crashes_longitude_latitude_not_null.loc[[0]]

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,crash_year,crash_month,crash_month_year,crash_day,crash_hour
0,06/09/2019,11:32,MANHATTAN,10012,40.72521,-73.99586,POINT (-73.99586 40.72521),EAST HOUSTON STREET,CROSBY STREET,NaN,...,Sedan,Taxi,NaN,NaN,NaN,2019,6,2019-06,9,11


In [9]:
def label_precinct(row):
    row_longitude = row['LONGITUDE']
    row_latitude = row['LATITUDE']
    row_point = Point(row_longitude, row_latitude)
    precinct_label = ""
    
    for index, precinct in df_precincts.iterrows():
        mp = MultiPolygon(precinct.geometry)
        if row_point.within(mp):
            #for debugging
            #print(' the point fits! ')
            #print(' at precinct: ' + precinct['precinct'])
            precinct_label = precinct['precinct']
            break
    
    return precinct_label

In [10]:
#test
label_precinct(df_crashes_longitude_latitude_not_null.loc[[0]])

'5'

In [11]:
4*multiprocessing.cpu_count()

48

In [12]:
#option 1
#setting npartitions doesn't work, since it has to be single processing with pandas apply...
import swifter
df_crashes_with_precincts = df_crashes_longitude_latitude_not_null.swifter.set_npartitions(36).apply(lambda row: label_precinct(row), axis=1)
df_crashes_with_precincts

c:\users\alexf\anaconda3\envs\nyc_crash_env\lib\site-packages\dask\dataframe\utils.py:15: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
c:\users\alexf\anaconda3\envs\nyc_crash_env\lib\site-packages\tqdm\std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


0            5
1           61
2           72
3          121
4           75
          ... 
1643286     63
1643287      5
1643288     83
1643289     28
1643291     20
Length: 1443867, dtype: object

In [ ]:
#option 2
#with multiprocessing.Pool(multiprocessing.cpu_count()) as pool:
#    df_crashes_with_precincts = pool.map(label_precinct, df_crashes_longitude_latitude_not_null)

In [ ]:
#option 3
#def parallelize_dataframe(df, func, n_cores=4):
#    df_split = np.array_split(df, n_cores)
#    pool = multiprocessing.Pool(n_cores)
#    df = pd.concat(pool.map(func, df_split))
#    pool.close()
#    pool.join()
#    return df

In [ ]:
#df_crashes_with_precincts = parallelize_dataframe(df_crashes_longitude_latitude_not_null, label_precinct)

In [16]:
#fuck, fix this 
type(df_crashes_with_precincts)
print(len(df_crashes_with_precincts))

1443867


In [17]:
df_total = df_crashes_longitude_latitude_not_null.copy()
df_total['precinct'] = df_crashes_with_precincts

In [18]:
df_total

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5,crash_year,crash_month,crash_month_year,crash_day,crash_hour,precinct
0,06/09/2019,11:32,MANHATTAN,10012,40.725210,-73.995860,POINT (-73.99586 40.72521),EAST HOUSTON STREET,CROSBY STREET,NaN,...,Taxi,NaN,NaN,NaN,2019,6,2019-06,9,11,5
1,06/28/2019,8:55,BROOKLYN,11235,40.586680,-73.945114,POINT (-73.945114 40.58668),VOORHIES AVENUE,EAST 24 STREET,NaN,...,NaN,NaN,NaN,NaN,2019,6,2019-06,28,8,61
2,06/18/2019,15:00,BROOKLYN,11232,40.670570,-73.997630,POINT (-73.99763 40.67057),NaN,NaN,435 HAMILTON AVENUE,...,Pick-up Truck,NaN,NaN,NaN,2019,6,2019-06,18,15,72
3,06/30/2019,22:30,NaN,NaN,40.617893,-74.164130,POINT (-74.16413 40.617893),GAULDY AVENUE,LONGDALE STREET,NaN,...,NaN,NaN,NaN,NaN,2019,6,2019-06,30,22,121
4,06/10/2019,19:20,BROOKLYN,11208,40.681355,-73.880220,POINT (-73.88022 40.681355),FULTON STREET,HALE AVENUE,NaN,...,Pick-up Truck,NaN,NaN,NaN,2019,6,2019-06,10,19,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1643286,09/18/2012,18:15,BROOKLYN,11234,40.626073,-73.923740,POINT (-73.9237396 40.6260734),AVENUE K,EAST 54 STREET,NaN,...,PASSENGER VEHICLE,NaN,NaN,NaN,2012,9,2012-09,18,18,63
1643287,09/22/2012,9:00,MANHATTAN,10012,40.720626,-73.994971,POINT (-73.9949711 40.7206259),ELIZABETH STREET,KENMARE STREET,NaN,...,PASSENGER VEHICLE,NaN,NaN,NaN,2012,9,2012-09,22,9,5
1643288,09/20/2012,17:15,BROOKLYN,11207,40.684513,-73.909380,POINT (-73.9093801 40.6845131),BUSHWICK AVENUE,MOFFAT STREET,NaN,...,SPORT UTILITY / STATION WAGON,NaN,NaN,NaN,2012,9,2012-09,20,17,83
1643289,09/19/2012,12:36,MANHATTAN,10026,40.805032,-73.956820,POINT (-73.9568204 40.8050324),MANHATTAN AVENUE,WEST 116 STREET,NaN,...,TAXI,NaN,NaN,NaN,2012,9,2012-09,19,12,28


In [23]:
#looks about right
df_total['precinct'].unique()

array(['5', '61', '72', '121', '75', '48', '23', '45', '77', '111', '115',
       '19', '90', '114', '94', '14', '120', '63', '122', '43', '52',
       '113', '102', '18', '83', '40', '105', '79', '66', '1', '67',
       '101', '81', '41', '10', '103', '47', '44', '108', '33', '7',
       '106', '20', '70', '24', '110', '6', '17', '46', '107', '76', '88',
       '71', '60', '69', '28', '25', '78', '109', '', '49', '73', '50',
       '9', '13', '26', '62', '112', '104', '34', '42', '68', '84', '30',
       '32', '100', '22', '123'], dtype=object)

In [24]:
df_total.to_csv('..\..\data\motor_vehicle_collisions_crashes_with_precinct.csv')